# BATTLE OF RESTAURANT LOCATIONS

*Bashir Abubakar* https://www.linkedin.com/in/bashir-abubakar-61935417b/

# Business Problem

A business owner of an African restaurant in Texas is looking to open a new branch in New York city. As a business intelligence analyst, I am going to help him with a suitable location for this new restaurant.

# Dataset

As the defined region is New York city. I will use the data sources from 'https://en.wikipedia.org/wiki/New_York_City' and 'https://en.wikipedia.org/wiki/Demographics_of_New_York_City' to get input of its population density and demographics data. 
Moreover, to explore the neighborhoods in New York, the dataset from 'https://cocl.us/new_york_dataset' and Foursquare API will also be used to solve and execute and business problem mentioned above.

# Methodology

First, discover the overall population in New York city by applying the dataset from Wikipedia.

In [30]:
# Import necessary packages
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes

import os
import folium # map rendering library
# Matplotlib and associated plotting modules

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mp
import re
import csv
%matplotlib inline
#import chart studio
!pip install chart-studio
!pip install cufflinks
from chart_studio.plotly import plot, iplot
from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly
import chart_studio
chart_studio.tools.set_credentials_file(username='bashman18', api_key='••••••••••')
init_notebook_mode(connected=True)

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.



Libraries imported.


Then, explore and prepare the data for the table of race in each jurisdiction

In [31]:
response_obj = requests.get('https://en.wikipedia.org/wiki/New_York_City').text
soup = BeautifulSoup(response_obj,'lxml')
Neighborhoods_NYC_Table = soup.find('table', {'class':'wikitable sortable'})

rows = Neighborhoods_NYC_Table.select("tbody > tr")[3:8]

boroughs = []
for row in rows:
    borough = {}
    tds = row.select('td')
    borough["borough"] = tds[0].text.strip()
    borough["county"] = tds[1].text.strip()
    borough["population"] = float(tds[2].text.strip().replace(",",""))
    borough["gdp_billions"] = float(tds[3].text.strip().replace(",",""))
    borough["gdp_per_capita"] = float(tds[4].text.strip().replace(",",""))
    borough["land_sqm"] = float(tds[5].text.strip().replace(",",""))
    borough["land_sqkm"] = float(tds[6].text.strip().replace(",",""))
    borough["persons_sqm"] = float(tds[7].text.strip().replace(",",""))
    borough["persons_sqkm"] = float(tds[8].text.strip().replace(",",""))
    
    boroughs.append(borough)

#print(boroughs)
boroughs_df=boroughs
pd.DataFrame(boroughs_df)

,borough,county,population,gdp_billions,gdp_per_capita,land_sqm,land_sqkm,persons_sqm,persons_sqkm
0,The Bronx,Bronx,1418207.0,42.695,30100.0,42.10,109.04,33867.0,13006.0
1,Brooklyn,Kings,2559903.0,91.559,35800.0,70.82,183.42,36147.0,13957.0
2,Manhattan,New York,1628706.0,600.244,368500.0,22.83,59.13,71341.0,27544.0
3,Queens,Queens,2253858.0,93.310,41400.0,108.53,281.09,20767.0,8018.0
4,Staten Island,Richmond,476143.0,14.514,30500.0,58.37,151.18,8157.0,3150.0


In [32]:
response_obj = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(response_obj,'lxml')
Population_Census_Table = soup.select_one('.wikitable:nth-of-type(5)') #use css selector to target correct table.

jurisdictions = []
rows = Population_Census_Table.select("tbody > tr")[3:8]
for row in rows:
    jurisdiction = {}
    tds = row.select('td')
    jurisdiction["jurisdiction"] = tds[0].text.strip()
    jurisdiction["population_census"] = tds[1].text.strip()
    jurisdiction["%_white"] = float(tds[2].text.strip().replace(",",""))
    jurisdiction["%_black_or_african_amercian"] = float(tds[3].text.strip().replace(",",""))
    jurisdiction["%_Asian"] = float(tds[4].text.strip().replace(",",""))
    jurisdiction["%_other"] = float(tds[5].text.strip().replace(",",""))
    jurisdiction["%_mixed_race"] = float(tds[6].text.strip().replace(",",""))
    jurisdiction["%_hispanic_latino_of_other_race"] = float(tds[7].text.strip().replace(",",""))
    jurisdiction["%_catholic"] = float(tds[10].text.strip().replace(",",""))
    jurisdiction["%_jewish"] = float(tds[12].text.strip().replace(",",""))
    jurisdictions.append(jurisdiction)

#print(jurisdictions)
jurisdictions_df=jurisdictions
pd.DataFrame(jurisdictions_df)

,jurisdiction,population_census,%_white,%_black_or_african_amercian,%_Asian,%_other,%_mixed_race,%_hispanic_latino_of_other_race,%_catholic,%_jewish
0,Queens,"2,229,379",44.1,20.0,17.6,12.3,6.1,25.0,37.0,5.0
1,Manhattan,"1,537,195",54.4,17.4,9.4,14.7,4.1,27.2,11.0,9.0
2,Bronx,"1,332,650",29.9,35.6,3.0,25.7,5.8,48.4,14.0,5.0
3,Staten Island,"443,728",77.6,9.7,5.7,4.3,2.7,12.1,11.0,5.0
4,NYC Total,"8,008,278",44.7,26.6,9.8,14.0,4.9,27.0,17.0,6.0


In [33]:
# Prepare the dataframe
df = pd.DataFrame(jurisdictions, columns=["jurisdiction","%_white", "%_black_or_african_amercian", "%_Asian", "%_other", "%_mixed_race", "%_hispanic_latino_of_other_race"])
df.head()

,jurisdiction,%_white,%_black_or_african_amercian,%_Asian,%_other,%_mixed_race,%_hispanic_latino_of_other_race
0,Queens,44.1,20.0,17.6,12.3,6.1,25.0
1,Manhattan,54.4,17.4,9.4,14.7,4.1,27.2
2,Bronx,29.9,35.6,3.0,25.7,5.8,48.4
3,Staten Island,77.6,9.7,5.7,4.3,2.7,12.1
4,NYC Total,44.7,26.6,9.8,14.0,4.9,27.0


We can see that Bronx has the the highest Black or African American ethnic population in New York with 35.6%, following Queens with the ratio of 20.0%.

# Moving to explore the neighbourhoods and consider the restaurant rating for each Vietnamese restaurants venues

I will use FourSquare API and the dataset from 'https://cocl.us/new_york_dataset' that contains list Boroughs, Neighbourhoods along with their latitude and longitude.

In [34]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude

Define a function to interact with FourSquare API and get top 100 venues within a radius of 1000 metres for a given latitude and longitude.

In [118]:
def get_venues(lat,lng):
    
    #set variables
    radius=1000
    LIMIT=100
    CLIENT_ID = 'PJSZT3QA54UA1VYW0VH4AYT30HWXXIJG0LFDRCTITDB3X5CD'# Foursquare ID, note there is a daily call quota limit 
    CLIENT_SECRET ='TLZDVDTEWZFYKDCJPICVTZC2PROSKC5H5FCCKVH1T1TXRCSZ' # Foursquare Secret, note there is a daily call quota it
    VERSION = '20180323' # Foursquare API version
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

Define a ranking function based on venue details with the elements including count , rating , tip counts for a given venue id.

In [116]:
def get_venue_details(venue_id):
        
    CLIENT_ID = 'PJSZT3QA54UA1VYW0VH4AYT30HWXXIJG0LFDRCTITDB3X5CD'# Foursquare ID, note there is a daily call quota limit 
    CLIENT_SECRET ='TLZDVDTEWZFYKDCJPICVTZC2PROSKC5H5FCCKVH1T1TXRCSZ' # Foursquare Secret, note there is a daily call quota it it
    VERSION = '20180323' # Foursquare API version
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

Define a funtion to get the New York city data such as Boroughs, Neighborhoods along with their latitude and longitude.

In [44]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

Convert New York dataset to a dataframe and display the first 5 rows.

In [45]:
new_york_data=get_new_york_data()
new_york_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [39]:
new_york_data.shape

(306, 4)

We then have the result of 306 neighborhoods to consider for the restaurant locations.

In [40]:
import plotly.express as px
new_york_data.groupby('Borough')['Neighborhood'].count().iplot(kind='bar',xTitle='Borough',
    yTitle='Neighborhood',
    mode='markers',
    title='Number of Neighborhood for each Borough in New York City')

In [119]:
# Prepare neighborhood list that contains african restaurant
column_names=['Borough', 'Neighborhood', 'ID','Name']
african_rest_ny=pd.DataFrame(columns=column_names)
count=1
for row in new_york_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    african_restaurants=venues[venues['Category']=='African Restaurant']   
    print('(',count,'/',len(new_york_data),')','African Restaurants in '+Neighborhood+', '+Borough+':'+str(len(african_restaurants)))
    for restaurant_detail in african_restaurants.values.tolist():
        id, name , category=restaurant_detail
        african_rest_ny = african_rest_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

( 1 / 306 ) African Restaurants in Wakefield, Bronx:0
( 2 / 306 ) African Restaurants in Co-op City, Bronx:0
( 3 / 306 ) African Restaurants in Eastchester, Bronx:0
( 4 / 306 ) African Restaurants in Fieldston, Bronx:0
( 5 / 306 ) African Restaurants in Riverdale, Bronx:0
( 6 / 306 ) African Restaurants in Kingsbridge, Bronx:0
( 7 / 306 ) African Restaurants in Marble Hill, Manhattan:0
( 8 / 306 ) African Restaurants in Woodlawn, Bronx:0
( 9 / 306 ) African Restaurants in Norwood, Bronx:0
( 10 / 306 ) African Restaurants in Williamsbridge, Bronx:0
( 11 / 306 ) African Restaurants in Baychester, Bronx:0
( 12 / 306 ) African Restaurants in Pelham Parkway, Bronx:0
( 13 / 306 ) African Restaurants in City Island, Bronx:0
( 14 / 306 ) African Restaurants in Bedford Park, Bronx:0
( 15 / 306 ) African Restaurants in University Heights, Bronx:2
( 16 / 306 ) African Restaurants in Morris Heights, Bronx:0
( 17 / 306 ) African Restaurants in Fordham, Bronx:1
( 18 / 306 ) African Restaurants in Ea

KeyError: 'groups'

In [120]:
african_rest_ny.head()

,Borough,Neighborhood,ID,Name
0,Bronx,University Heights,4c2f8b1b7cc0c9b6a1eaeb9a,Accra Resturant
1,Bronx,University Heights,4cb2beaadedbef3bca7c9d40,Papaye Restaurant
2,Bronx,Fordham,4cb2beaadedbef3bca7c9d40,Papaye Restaurant
3,Bronx,Belmont,4cb2beaadedbef3bca7c9d40,Papaye Restaurant


In [121]:
african_rest_ny.shape

(4, 4)

In [122]:
import plotly.express as px
african_rest_ny.groupby('Borough')['ID'].count().iplot(kind='bar',xTitle='Borough',
    yTitle='No.of African Restaurant',
    mode='markers',
    title='Number of African Restaurants in NYC by Borough')

Based on the chart, Bronx has the highest number of African restaurants.

In [123]:
african_rest_ny[african_rest_ny['Neighborhood']=='University Heights']

,Borough,Neighborhood,ID,Name
0,Bronx,University Heights,4c2f8b1b7cc0c9b6a1eaeb9a,Accra Resturant
1,Bronx,University Heights,4cb2beaadedbef3bca7c9d40,Papaye Restaurant


In [125]:
import plotly.express as px
african_rest_ny.groupby('Neighborhood')['ID'].count().iplot(kind='bar',xTitle='Neighborhood',
    yTitle='No.of African Restaurants',
    mode='markers',
    title='Number of African Restaurants in NYC by Neighbourhood')

University Heights in Bronx still has the highest number of African restaurants toghether with Fordham and Belmont.

In [126]:
# Prepare neighborhood list that contains african restaurants
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Tips']
african_rest_stats_ny=pd.DataFrame(columns=column_names)
count=1


for row in african_rest_ny.values.tolist():
    Borough,Neighborhood,ID,Name=row

    # prepare neighborhood list that contains vietnamese restaurants
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Tips']
african_rest_stats_ny=pd.DataFrame(columns=column_names)
count=1


for row in african_rest_ny.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details=get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except (IndexError, KeyError) as e:
        print('No data available for id=',ID)
        # we will assign 0 value for these resturants as they may have been 
        #recently opened or details does not exist in FourSquare Database
        id,name,likes,rating,tips=[0]*5
    print('(',count,'/',len(african_rest_ny),')','processed')
    african_rest_stats_ny = african_rest_stats_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
    count+=1

                         ID             Name  Likes  Rating  Tips
0  4c2f8b1b7cc0c9b6a1eaeb9a  Accra Resturant      9     7.3     3
( 1 / 4 ) processed
                         ID               Name  Likes  Rating  Tips
0  4cb2beaadedbef3bca7c9d40  Papaye Restaurant     11     7.5     3
( 2 / 4 ) processed
                         ID               Name  Likes  Rating  Tips
0  4cb2beaadedbef3bca7c9d40  Papaye Restaurant     11     7.5     3
( 3 / 4 ) processed
                         ID               Name  Likes  Rating  Tips
0  4cb2beaadedbef3bca7c9d40  Papaye Restaurant     11     7.5     3
( 4 / 4 ) processed


In [127]:
african_rest_stats_ny.head()

,Borough,Neighborhood,ID,Name,Likes,Rating,Tips
0,Bronx,University Heights,4c2f8b1b7cc0c9b6a1eaeb9a,Accra Resturant,9,7.3,3
1,Bronx,University Heights,4cb2beaadedbef3bca7c9d40,Papaye Restaurant,11,7.5,3
2,Bronx,Fordham,4cb2beaadedbef3bca7c9d40,Papaye Restaurant,11,7.5,3
3,Bronx,Belmont,4cb2beaadedbef3bca7c9d40,Papaye Restaurant,11,7.5,3


In [128]:
african_rest_stats_ny.shape

(4, 7)

In [129]:
african_rest_stats_ny.to_csv('african_rest_stats_ny.csv', index=False)

In [130]:
african_rest_stats_ny_csv=pd.read_csv('african_rest_stats_ny.csv')

In [131]:
african_rest_stats_ny_csv.shape

(4, 7)

In [132]:
african_rest_stats_ny_csv.head()

,Borough,Neighborhood,ID,Name,Likes,Rating,Tips
0,Bronx,University Heights,4c2f8b1b7cc0c9b6a1eaeb9a,Accra Resturant,9,7.3,3
1,Bronx,University Heights,4cb2beaadedbef3bca7c9d40,Papaye Restaurant,11,7.5,3
2,Bronx,Fordham,4cb2beaadedbef3bca7c9d40,Papaye Restaurant,11,7.5,3
3,Bronx,Belmont,4cb2beaadedbef3bca7c9d40,Papaye Restaurant,11,7.5,3


In [133]:
african_rest_stats_ny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
Borough         4 non-null object
Neighborhood    4 non-null object
ID              4 non-null object
Name            4 non-null object
Likes           4 non-null object
Rating          4 non-null float64
Tips            4 non-null object
dtypes: float64(1), object(6)
memory usage: 352.0+ bytes


In [134]:
african_rest_stats_ny['Likes']=african_rest_stats_ny['Likes'].astype('float64')

In [135]:
african_rest_stats_ny['Tips']=african_rest_stats_ny['Tips'].astype('float64')

In [136]:
african_rest_stats_ny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
Borough         4 non-null object
Neighborhood    4 non-null object
ID              4 non-null object
Name            4 non-null object
Likes           4 non-null float64
Rating          4 non-null float64
Tips            4 non-null float64
dtypes: float64(3), object(4)
memory usage: 352.0+ bytes


In [140]:
# Restaurants with maximum Likes
african_rest_stats_ny.iloc[african_rest_stats_ny['Likes'].idxmax()]

Borough                            Bronx
Neighborhood          University Heights
ID              4cb2beaadedbef3bca7c9d40
Name                   Papaye Restaurant
Likes                                 11
Rating                               7.5
Tips                                   3
Name: 1, dtype: object

In [141]:
# Restaurants with maximum Rating
african_rest_stats_ny.iloc[african_rest_stats_ny['Rating'].idxmax()]

Borough                            Bronx
Neighborhood          University Heights
ID              4cb2beaadedbef3bca7c9d40
Name                   Papaye Restaurant
Likes                                 11
Rating                               7.5
Tips                                   3
Name: 1, dtype: object

In [143]:
# Restaurants with maximum Tips
african_rest_stats_ny.iloc[african_rest_stats_ny['Tips'].idxmax()]

Borough                            Bronx
Neighborhood          University Heights
ID              4c2f8b1b7cc0c9b6a1eaeb9a
Name                     Accra Resturant
Likes                                  9
Rating                               7.3
Tips                                   3
Name: 0, dtype: object

In [144]:
ny_neighborhood_stats=african_rest_stats_ny.groupby('Neighborhood',as_index=False).mean()[['Neighborhood','Rating']]
ny_neighborhood_stats.columns=['Neighborhood','Average Rating']

In [145]:
ny_neighborhood_stats.sort_values(['Average Rating'],ascending=False).head(10)

,Neighborhood,Average Rating
0,Belmont,7.5
1,Fordham,7.5
2,University Heights,7.4


In [146]:
ny_borough_stats=african_rest_stats_ny.groupby('Borough',as_index=False).mean()[['Borough','Rating']]
ny_borough_stats.columns=['Borough','Average Rating']

In [147]:
ny_borough_stats.sort_values(['Average Rating'],ascending=False).head()

,Borough,Average Rating
0,Bronx,7.45


In [157]:
import plotly.express as px
african_rest_stats_ny.groupby('Neighborhood').mean()['Rating'].iplot(kind='bar',xTitle='Neighborhood',
    yTitle='Average Rating',
    mode='markers',
    title='Average rating of African Restaurant in each  NYC Borough')

Bronx also has the highest average rating for African restaurants.

In [151]:
# Visualize the neighborhoods with average rating greater or equal 7.0 
ny_neighborhood_stats=ny_neighborhood_stats[ny_neighborhood_stats['Average Rating']>=7.0]
ny_neighborhood_stats

,Neighborhood,Average Rating
0,Belmont,7.5
1,Fordham,7.5
2,University Heights,7.4


In [152]:
ny_neighborhood_stats=pd.merge(ny_neighborhood_stats,new_york_data, on='Neighborhood')
ny_neighborhood_stats=ny_neighborhood_stats[['Borough','Neighborhood','Latitude','Longitude','Average Rating']]
ny_neighborhood_stats.sort_values(['Average Rating'],ascending=False).head(10)

,Borough,Neighborhood,Latitude,Longitude,Average Rating
0,Bronx,Belmont,40.857277,-73.888452,7.5
1,Bronx,Fordham,40.860997,-73.896427,7.5
2,Bronx,University Heights,40.855727,-73.910416,7.4


In [153]:
# create map and display it
ny_map = folium.Map(location=(40.693943, -73.985880), zoom_start=12)

In [154]:
# Instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in ny_neighborhood_stats[['Latitude','Longitude']].values:
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

In [155]:
ny_neighborhood_stats['Label']=ny_neighborhood_stats['Neighborhood']+', '+ny_neighborhood_stats['Borough']+'('+ny_neighborhood_stats['Average Rating'].map(str)+')'

In [156]:
# Put on map best neighbourhoods in New York that has highest average rating for VNmese restaurants

# add pop-up text to each marker on the map
for lat, lng, label in ny_neighborhood_stats[['Latitude','Longitude','Label']].values:
    folium.Marker([lat, lng], popup=label).add_to(ny_map)        
# add incidents to map
ny_map.add_child(incidents)

# Result

- Brooklyn is beyond the top highest African American ethnic population in New York city.
- Bronx has the highest number of African restaurants especially in University Heights with the most crowded number.
- Belmont, Fordham, University heights in Bronx are among the best neighborhoods for African cuisine with highest rating.
- Papaye Restaurant has the most likes in Bronx.

# Discussion & Recommendation

Bronx in New York city will be the best choice for the African restaurant owner because it is the fourth most populous urban area in New York City and in the top list of African ethnic population in New York. Moreover, it possesses of the top rated African restaurants located in that area.